In [1]:
import pandas as pd

import sys
import os
sys.path.insert(0, "/home/jovyan/.local/share/virtualenvs/ptls-experiments-w-dEu3oS/lib/python3.8/site-packages")
os.environ["OMP_NUM_THREADS"] = "4"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer
import torch
from transformers import DebertaV2ForMaskedLM, DebertaV2Config
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model

tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-large')

dataset = load_dataset('wikipedia', '20220301.en', split='train[:1%]')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=256)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

/home/jovyan/.local/share/virtualenvs/ptls-experiments-w-dEu3oS/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/home/jovyan/.local/share/virtualenvs/ptls-experiments-w-dEu3oS/lib/python3.8/site-packages/datasets/load.py:1486: FutureWarning: The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to 

In [3]:
def count_parameters(model):
    total_params = 0
    trainable_params = 0
    non_trainable_params = 0

    for param in model.parameters():
        num_params = param.numel()
        total_params += num_params
        if param.requires_grad:
            trainable_params += num_params
        else:
            non_trainable_params += num_params

    print(total_params, trainable_params, non_trainable_params)

In [4]:
warm_start_args = TrainingArguments(
    output_dir='./results',
    # max_steps=,
    num_train_epochs=5,
    per_device_train_batch_size=32,
    warmup_steps=100,
    weight_decay=0.01,      
    logging_dir='./logs',
    logging_steps=100,
    # save_strategy='steps',
    # save_steps=200,
    lr_scheduler_type='cosine', 
    report_to="wandb",
    save_strategy='no',
)

In [ ]:
from transformers import Trainer, TrainingArguments
import torch

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
config = DebertaV2Config.from_pretrained('microsoft/deberta-v3-large')
model = DebertaV2ForMaskedLM.from_pretrained('microsoft/deberta-v3-large')

class CustomTrainer(Trainer):
    def training_step(self, model, inputs):
        loss = super().training_step(model, inputs)
        step = self.state.global_step

        lora_config = LoraConfig(
            r=256, 
            lora_alpha=32,
            lora_dropout=0.1,
            bias="none",
            inference_mode=False
        )

        if step == 500:
            self.model = get_peft_model(self.model, lora_config)
            self.optimizer = torch.optim.SGD(self.model.parameters(), lr=1e-5)
            self.optimizer.zero_grad(set_to_none=True)

        if step % 1000 == 0 and step > 100:
            self.model = self.model.merge_and_unload()
            self.model = get_peft_model(self.model, lora_config)
            self.optimizer = torch.optim.SGD(self.model.parameters(), lr=1e-5)
            self.optimizer.zero_grad(set_to_none=True)
        return loss
    
lora_config = LoraConfig(
    r=256, 
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    inference_mode=False
)

    
model = get_peft_model(model, lora_config)


trainer = Trainer(
    model=model,
    args=warm_start_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator
)
trainer.train()

/home/jovyan/.local/share/virtualenvs/ptls-experiments-w-dEu3oS/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/jovyan/.local/share/virtualenvs/ptls-experiments-w-dEu3oS/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.trans

/home/jovyan/.local/share/virtualenvs/ptls-experiments-w-dEu3oS/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Step,Training Loss
100,12.072200
200,8.529300
300,7.782900
400,7.446500
500,7.244700
600,7.070900
700,6.919600
800,6.774700
900,6.688500
1000,6.603300


In [2]:
# from datasets import load_dataset
# from transformers import AutoTokenizer, DebertaV2ForSequenceClassification, DebertaV2Config
# from transformers import Trainer, TrainingArguments
# from peft import LoraConfig, get_peft_model

# tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-large')
# dataset = load_dataset("glue", "cola", split='train')

# def tokenize_function(examples):
#     return tokenizer(examples['sentence'], padding="max_length", truncation=True, max_length=128)

# tokenized_datasets = dataset.map(tokenize_function, batched=True)

# config = DebertaV2Config.from_pretrained('microsoft/deberta-v3-large', num_labels=2)
# model = DebertaV2ForSequenceClassification(config)

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DebertaV2ForSequenceClassification, DebertaV2Config
from transformers import Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-large')
dataset = load_dataset("glue", "cola", split='train')

def tokenize_and_encode(examples):
    tokenized_inputs = tokenizer(examples['sentence'], padding="max_length", truncation=True, max_length=64)
    tokenized_inputs['labels'] = examples['label']
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_encode, batched=True)
config = DebertaV2Config.from_pretrained('microsoft/deberta-v3-large', num_labels=2)
model = DebertaV2ForSequenceClassification(config)

/home/jovyan/.local/share/virtualenvs/ptls-experiments-w-dEu3oS/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map:   0%|          | 0/8551 [00:00<?, ? examples/s]

/home/jovyan/.local/share/virtualenvs/ptls-experiments-w-dEu3oS/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
warm_start_args = TrainingArguments(
    output_dir='./results',
    # max_steps=,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    warmup_steps=100,
    weight_decay=0.01,      
    logging_dir='./logs',
    logging_steps=200,
    save_strategy='steps',
    save_steps=200,
    lr_scheduler_type='cosine', 
)

In [4]:
from transformers import Trainer, TrainingArguments
import torch
class CustomTrainer(Trainer):
    def training_step(self, model, inputs):
        outputs = model(**inputs)
        loss = outputs.loss
        step = self.state.global_step

        lora_config = LoraConfig(
            r=256, 
            lora_alpha=32,
            lora_dropout=0.1,
            bias="none",
            inference_mode=False
        )
        torch.cuda.empty_cache()
        if step == 50:
            self.model = get_peft_model(self.model, lora_config)
            self.reset_optimizer()

        if step % 1000 == 0 and step > 100:
            self.model = self.model.merge_and_unload()
            self.model = get_peft_model(self.model, lora_config)
            self.reset_optimizer()

        return loss
    
trainer = Trainer(
    model=model,
    args=warm_start_args,
    train_dataset=tokenized_datasets,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: ozoloevwork (omarzoloev). Use `wandb login --relogin` to force relogin


Step,Training Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 79.15 GiB total capacity; 77.95 GiB already allocated; 12.38 MiB free; 77.96 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
def warm_start(
    model,
    args,
    dataset,
    data_collator,
):
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=dataset,
        data_collator=data_collator
    )
    trainer.train()
    
    return model

def relora(
    model,
    args,
    dataset,
    data_collator,
    relora_step,
):
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=dataset,
        data_collator=data_collator
    )
    if relora_step != 200:
        trainer.train(resume_from_checkpoint=f"./results/checkpoint-{relora_step}")
    else:
        trainer.train()
    
    return model

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
config = DebertaV2Config.from_pretrained('microsoft/deberta-v3-small')
model = DebertaV2ForMaskedLM(config)

warm_start_args = TrainingArguments(
    output_dir='./results',
    max_steps=200,
    per_device_train_batch_size=32,
    warmup_steps=200,
    weight_decay=0.01,      
    logging_dir='./logs',
    logging_steps=50,
    save_strategy='steps',
    save_steps=50,
)

model = warm_start(
    model,
    warm_start_args,
    tokenized_datasets, 
    data_collator,
)

lora_config = LoraConfig(
    r=128, 
    lora_alpha=16,
    lora_dropout=0.01,
    bias="none",
    inference_mode=False
)
print("warm start done")
i = 0
for relora_step in range(200, 500, 50):
    relora_args = TrainingArguments(
        output_dir='./results',             
        max_steps=50 + i,
        per_device_train_batch_size=32,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=50,
        save_strategy='steps',
        save_steps=50,
    )
    model = get_peft_model(model, lora_config)
    model.save_pretrained(f"./results/checkpoint-{relora_step + 50}")
    
    model = relora(
        model,
        relora_args,
        tokenized_datasets, 
        data_collator,
        relora_step,
    )
    i += 50
    model = model.merge_and_unload()

In [43]:
model = relora(
    model,
    relora_args,
    tokenized_datasets, 
    data_collator,
    relora_step,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset,
    data_collator=data_collator
)
if relora_step != 200:
    trainer.train(resume_from_checkpoint=f"./results/checkpoint-{relora_step}")
else:
    trainer.train()

i += 50
model = model.merge_and_unload()

max_steps is given, it will override any value given in num_train_epochs
There were missing keys in the checkpoint model loaded: ['base_model.model.deberta.embeddings.word_embeddings.weight', 'base_model.model.deberta.embeddings.LayerNorm.weight', 'base_model.model.deberta.embeddings.LayerNorm.bias', 'base_model.model.deberta.encoder.layer.0.attention.self.query_proj.base_layer.weight', 'base_model.model.deberta.encoder.layer.0.attention.self.query_proj.base_layer.bias', 'base_model.model.deberta.encoder.layer.0.attention.self.query_proj.lora_A.default.weight', 'base_model.model.deberta.encoder.layer.0.attention.self.query_proj.lora_B.default.weight', 'base_model.model.deberta.encoder.layer.0.attention.self.key_proj.weight', 'base_model.model.deberta.encoder.layer.0.attention.self.key_proj.bias', 'base_model.model.deberta.encoder.layer.0.attention.self.value_proj.base_layer.weight', 'base_model.model.deberta.encoder.layer.0.attention.self.value_proj.base_layer.bias', 'base_model.model.

ValueError: loaded state dict contains a parameter group that doesn't match the size of optimizer's group

In [ ]:
# config = DebertaV2Config.from_pretrained('microsoft/deberta-v3-small')
# model = DebertaV2ForMaskedLM(config)

# # training_args = TrainingArguments(
# #     output_dir='./results',           
# #     num_train_epochs=1,  
# #     # max_steps=50,
# #     per_device_train_batch_size=32,   
# #     warmup_steps=1000,               
# #     weight_decay=0.01,             
# #     logging_dir='./logs',
# #     logging_steps=50,
# #     save_strategy='steps',
# #     save_steps=50, 
# # )
# from peft import LoraConfig, get_peft_model

# warm_start_args = TrainingArguments(
#     output_dir='./results',           
#     # num_train_epochs=1,  
#     max_steps=200,
#     per_device_train_batch_size=32,   
#     warmup_steps=200,               
#     weight_decay=0.01,      
#     logging_dir='./logs',
#     logging_steps=50,
#     save_strategy='steps',
#     save_steps=50, 
# )

# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

# trainer = Trainer(
#     model=model,
#     args=warm_start_args,
#     train_dataset=tokenized_datasets,
#     data_collator=data_collator
# )
 
# trainer.train()

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=128, 
    lora_alpha=16,
    lora_dropout=0.01,
    bias="none",
    inference_mode=False
)

model = get_peft_model(model, lora_config)

In [23]:
relora_args = TrainingArguments(
        output_dir='./results',           
        # num_train_epochs=1,  
        max_steps=50,
        per_device_train_batch_size=32,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=50,
        save_strategy='steps',
        save_steps=50,
)

trainer = Trainer(
    model=model,
    args=relora_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator
)
    
trainer.train()

max_steps is given, it will override any value given in num_train_epochs
/home/jovyan/.local/share/virtualenvs/ptls-experiments-w-dEu3oS/lib/python3.8/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in  - will assume that the vocabulary was not modified.
  warnings.warn(


Step,Training Loss
50,7.246800


TrainOutput(global_step=50, training_loss=7.2467626953125, metrics={'train_runtime': 21.6683, 'train_samples_per_second': 73.841, 'train_steps_per_second': 2.308, 'total_flos': 112090713292800.0, 'train_loss': 7.2467626953125, 'epoch': 0.024764735017335313})

In [ ]:
for i in range():

    relora_args = TrainingArguments(
        output_dir='./results',           
        # num_train_epochs=1,  
        max_steps=100,
        per_device_train_batch_size=32,   
        weight_decay=0.01,      
        logging_dir='./logs',
        logging_steps=50,
        save_strategy='steps',
        save_steps=50, 
    )
    
trainer.train(resume_from_checkpoint="results/checkpoint-200")

In [11]:
# resume_training_args = TrainingArguments(
#     output_dir='./results',           
#     num_train_epochs=1,  
#     # max_steps=50,
#     per_device_train_batch_size=32,   
#     warmup_steps=1000,               
#     weight_decay=0.01,             
#     logging_dir='./logs',
#     logging_steps=50,
#     save_strategy='steps',
#     save_steps=50, 
#     # resume_from_checkpoint='./results/checkpoint-100'  # Указание последнего checkpoint
# )

trainer.train(resume_from_checkpoint="results/checkpoint-350")

There were missing keys in the checkpoint model loaded: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias'].


Step,Training Loss
400,7.914600


KeyboardInterrupt: 

In [ ]:
for i in range(0, 10):
    

In [25]:
optimizer = trainer.optimizer
for group in optimizer.param_groups:
    for p in group["params"]:
        param_state = optimizer.state[p]
        param_state["exp_avg"].zero_()
        param_state["exp_avg_sq"].zero_()

In [6]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=128, 
    lora_alpha=16,
    lora_dropout=0.01,
    bias="none",
    inference_mode=False
)

model = get_peft_model(model, lora_config)

In [8]:
training_args = TrainingArguments(
    output_dir='./results',
    # num_train_steps=500,
    max_steps=100,
    per_device_train_batch_size=32,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator,
)

trainer.train()

max_steps is given, it will override any value given in num_train_epochs
/home/jovyan/.local/share/virtualenvs/ptls-experiments-w-dEu3oS/lib/python3.8/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in  - will assume that the vocabulary was not modified.
  warnings.warn(


Step,Training Loss
50,7.122600
100,7.157700
150,7.116300
200,7.124800


KeyboardInterrupt: 

In [26]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(r=128, lora_alpha=16, lora_dropout=0.01, bias="none", inference_mode=False)
model = get_peft_model(model, lora_config)

In [ ]:
num_lora_reset = 1
for num_lora_resets

In [62]:
training_args = TrainingArguments(
    output_dir='./results',           
    # num_train_steps=500,     
    max_steps=100,
    per_device_train_batch_size=32,   
    # warmup_steps=500,               
    weight_decay=0.01,             
    logging_dir='./logs',           
    logging_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator

)

trainer.train()

max_steps is given, it will override any value given in num_train_epochs
/home/jovyan/.local/share/virtualenvs/ptls-experiments-w-dEu3oS/lib/python3.8/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in  - will assume that the vocabulary was not modified.
  warnings.warn(


Step,Training Loss
100,4.847300
200,4.830600
300,4.778200
400,4.781000
500,4.783700


TrainOutput(global_step=500, training_loss=4.804149475097656, metrics={'train_runtime': 428.1897, 'train_samples_per_second': 37.367, 'train_steps_per_second': 1.168, 'total_flos': 2241814265856000.0, 'train_loss': 4.804149475097656, 'epoch': 0.24764735017335315})

In [59]:
optimizer = trainer.optimizer

In [60]:
for group in optimizer.param_groups:
    for p in group["params"]:
        param_state = optimizer.state[p]
        # param_state["exp_avg"] = torch.zeros_like(p.data)
        # param_state["exp_avg_sq"] = torch.zeros_like(p.data)

In [61]:
param_state

{'step': tensor(3.),
 'exp_avg': tensor([[ 1.6968e-06,  2.2630e-06,  1.7404e-07,  ..., -5.5712e-07,
           9.8056e-07, -7.5918e-07],
         [-1.6801e-06, -5.6132e-07, -2.9442e-07,  ..., -8.4616e-07,
           1.2062e-06, -8.2182e-07],
         [-1.0606e-06, -8.9447e-07, -1.4829e-06,  ..., -6.5035e-08,
           3.9825e-07, -2.2794e-06],
         ...,
         [-1.3874e-06, -5.7455e-06, -1.1686e-06,  ...,  2.9414e-07,
           4.2116e-08,  3.9215e-06],
         [ 1.6769e-06,  1.3099e-06,  6.4355e-07,  ..., -1.9402e-06,
           2.1443e-06, -6.3585e-06],
         [-1.3323e-06, -5.2628e-07, -2.9180e-06,  ...,  2.4726e-06,
           4.7547e-06, -5.7350e-06]], device='cuda:0'),
 'exp_avg_sq': tensor([[1.3105e-13, 2.3313e-13, 3.8195e-13,  ..., 7.9111e-14, 8.4746e-14,
          1.0136e-12],
         [3.8909e-13, 7.2568e-13, 1.7258e-13,  ..., 3.0338e-13, 1.7797e-13,
          7.4109e-14],
         [9.1168e-14, 2.0220e-13, 7.1618e-13,  ..., 2.2738e-13, 9.7842e-14,
          3.7203e

In [40]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.1)

In [41]:
for group in optimizer.param_groups:
    for p in group["params"]:
        param_state = optimizer.state[p]
        param_state["exp_avg"] = torch.zeros_like(p.data)
        param_state["exp_avg_sq"] = torch.zeros_like(p.data)

In [7]:
def count_parameters(model):
    total_params = 0
    trainable_params = 0
    non_trainable_params = 0

    for param in model.parameters():
        num_params = param.numel()
        total_params += num_params
        if param.requires_grad:
            trainable_params += num_params
        else:
            non_trainable_params += num_params

    print(total_params, trainable_params, non_trainable_params)
count_parameters(model)

144383844 2359296 142024548
